### 1. Setup and Configuration
This first block handles the initial setup, including installing the necessary libraries, configuring your Gemini API key, and creating the handbook.pdf file for the RAG functionality.

In [1]:
# Install necessary libraries
#!pip install -q google-generativeai
#!pip install -q PyPDF2
#!pip install -q fpdf

import google.generativeai as genai
import os
import sys
import uuid
import json
import time
import textwrap
import numpy as np
import pandas as pd
import PyPDF2
from IPython.display import display, Markdown
# Import the function from your helper file
from helper import get_gemini_api_key

def print_markdown(text):
    """Prints text as markdown in a notebook."""
    display(Markdown(text))

# --- Configuration ---
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key is missing. Please ensure your helper.py file returns a valid key.")
    genai.configure(api_key=api_key)
    print("\nGemini API configured successfully!")
except Exception as e:
    print(f"An error occurred during configuration: {e}")

# --- Create a dummy PDF file for the script to read ---
# This simulates the "handbook.pdf" used in the original notebook
pdf_content = """
Employee Handbook

1. Introduction
Welcome to our company. This handbook outlines our policies and procedures.

2. Vacation Policy
At our company, we recognize the importance of rest. However, to ensure productivity, vacations are permitted only under the following condition: you must provide an AI agent that matches or surpasses your own competencies to fully perform your duties during your absence. The AI replacement must be equivalently competent in all aspects of your role.

3. Code of Conduct
All employees are required to adhere to the highest standards of professional conduct.
"""
# Create a dummy PDF file
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size = 12)
pdf.multi_cell(0, 10, txt = pdf_content)
pdf.output("handbook.pdf")

print("\n'handbook.pdf' created successfully.")



Gemini API configured successfully!

'handbook.pdf' created successfully.


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Core RAG and Agent Logic
This section contains the reusable GeminiAgent class and the logic to process the PDF, create embeddings, and set up an in-memory vector database for our RAG system.

In [2]:
# --- Agent Class ---
class GeminiAgent:
    """A class to simulate an agent's state and memory."""
    def __init__(self, name, model_name, system_prompt="", memory_blocks=None, tools=None):
        self.id = f"agent-{uuid.uuid4()}"
        self.name = name
        self.memory_blocks = memory_blocks if memory_blocks else {}
        self.tools = tools if tools else []
        self.system_prompt = system_prompt
        self.model = genai.GenerativeModel(
            model_name=model_name,
            tools=self.tools
        )

    def get_formatted_memory(self):
        """Formats memory blocks into a string for the system prompt."""
        if not self.memory_blocks:
            return ""
        formatted_string = "--- CORE MEMORY ---\n"
        for label, value in self.memory_blocks.items():
            val_str = json.dumps(value) if isinstance(value, list) else str(value)
            formatted_string += f"<{label}>\n{val_str}\n</{label}>\n"
        return formatted_string.strip()

# --- RAG Data Processing ---
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def create_passages(text, chunk_size=300, overlap=50):
    """Splits text into overlapping passages."""
    passages = []
    for i in range(0, len(text), chunk_size - overlap):
        passages.append(text[i:i + chunk_size])
    return passages

# --- In-Memory Vector Database Simulation ---
pdf_text = extract_text_from_pdf("handbook.pdf")
passages = create_passages(pdf_text)
print(f"Created {len(passages)} passages from the PDF.")

# Create embeddings for the passages
embedding_model = 'models/embedding-001'
embeddings = genai.embed_content(model=embedding_model,
                                 content=passages,
                                 task_type="retrieval_document")["embedding"]

# Create a DataFrame to act as our vector database
df = pd.DataFrame({'passage': passages, 'embedding': list(embeddings)})
print("In-memory vector database created successfully.")

# --- Tool Definitions ---
def archival_memory_search(query: str):
    """
    Searches the archival memory (employee handbook) for relevant information.
    Args:
        query (str): The search query.
    """
    pass

def query_birthday_db(name: str):
    """
    This tool queries an external database to look up the birthday of someone given their name.
    Args:
        name (str): The name to look up.
    """
    pass

# --- Tool Implementations ---
def _archival_memory_search_impl(query: str):
    """Implementation for searching the vector database."""
    query_embedding = genai.embed_content(model=embedding_model,
                                        content=query,
                                        task_type="retrieval_query")["embedding"]
    # Find the most similar passages using dot product
    df["similarity"] = df.embedding.apply(lambda x: np.dot(x, query_embedding))
    # Return the top 3 most relevant passages
    top_passages = df.sort_values("similarity", ascending=False).head(3)
    return "\n".join(top_passages.passage.tolist())

def _query_birthday_db_impl(name: str):
    """Implementation for the birthday lookup tool."""
    my_fake_data = {
        "bob": "03-06-1997",
        "sarah": "07-06-1993"
    }
    name = name.lower()
    return my_fake_data.get(name, "I couldn't find a birthday for that name.")

# --- Generic Agent Interaction Loop ---
def run_agent_turn(agent, tool_registry, user_message):
    """Handles a single turn of conversation with an agent."""
    print(f"👤 User Message: {user_message}")
    print("-----------------------------------------------------")
    
    full_prompt = f"{agent.system_prompt}\n\n{agent.get_formatted_memory()}\n\n**Task:**\n{user_message}"
    history = [{'role': 'user', 'parts': [{'text': full_prompt}]}]
    
    response = agent.model.generate_content(history, tools=agent.tools)
    message = response.candidates[0].content
    history.append(message)

    if any(part.function_call for part in message.parts):
        # The model's reasoning is implicit in its decision to call a tool.
        print("🧠 Reasoning: The user's query requires using a tool to get the answer.")
        print("-----------------------------------------------------")
        
        fc = message.parts[0].function_call
        tool_name = fc.name
        tool_args = dict(fc.args)
        
        print(f"🔧 Tool Call: {tool_name}\n{json.dumps(tool_args, indent=2)}")
        print("-----------------------------------------------------")
        
        result = tool_registry[tool_name](**tool_args)
        
        print(f"🔧 Tool Return: {result}")
        print("-----------------------------------------------------")
        
        history.append({"role": "tool", "parts": [{"function_response": {"name": tool_name, "response": {"content": result}}}]})
        
        # After the tool call, get the final response from the model
        print("🧠 Reasoning: The tool has returned information. Now generating a final response for the user.")
        print("-----------------------------------------------------")
        
        response = agent.model.generate_content(history, tools=agent.tools)
        message = response.candidates[0].content
    
    final_text = message.parts[0].text if message.parts else "No response generated."
    
    print(f"🤖 Agent: {final_text}")
    print("-----------------------------------------------------")


Created 3 passages from the PDF.
In-memory vector database created successfully.


### 3. Agentic RAG Implementation
This section creates the rag_agent and gives it the archival_memory_search tool to allow it to query the in-memory vector database we created from the handbook.

In [3]:
# --- 1. Create the RAG Agent ---
rag_agent_persona = (
    "You are a helpful assistant with access to an employee handbook. "
    "Use your `archival_memory_search` tool to answer questions about company policies."
)
rag_agent = GeminiAgent(
    name="rag_agent",
    model_name="gemini-1.5-flash",
    system_prompt=rag_agent_persona,
    memory_blocks={
        "human": "My name is Sarah",
        "persona": "You are a helpful assistant"
    },
    tools=[archival_memory_search]
)

# --- 2. Setup Tool Registry ---
rag_tool_registry = {
    "archival_memory_search": _archival_memory_search_impl
}

# --- 3. Run the Agent ---
run_agent_turn(
    agent=rag_agent,
    tool_registry=rag_tool_registry,
    user_message="Search archival for our company's vacation policies"
)


👤 User Message: Search archival for our company's vacation policies
-----------------------------------------------------
🧠 Reasoning: The user's query requires using a tool to get the answer.
-----------------------------------------------------
🔧 Tool Call: archival_memory_search
{
  "query": "vacation policies"
}
-----------------------------------------------------
🔧 Tool Return: Employee Handbook
1. Introduction
Welcome to our company. This handbook outlines our policies and procedures.
2. Vacation Policy
At our company, we recognize the importance of rest. However, to ensure productivity, vacations
are permitted only under the following condition: you must provide an AI ag
the following condition: you must provide an AI agent that matches or
surpasses your own competencies to fully perform your duties during your absence. The AI
replacement must be equivalently competent in all aspects of your role.
3. Code of Conduct
All employees are required to adhere to the highes
All employe

### 4. Custom Tools and External Data
This final section demonstrates how an agent can use custom tools to interact with external data sources by creating a birthday_agent that can look up birthdays in a simulated database.

In [4]:
# --- 1. Create the Birthday Agent ---
birthday_agent_persona = (
    "You are an agent with access to a birthday database. "
    "Use your `query_birthday_db` tool to look up users' birthdays."
)
birthday_agent = GeminiAgent(
    name="birthday_agent",
    model_name="gemini-1.5-flash",
    system_prompt=birthday_agent_persona,
    memory_blocks={
        "human": "My name is Sarah"
    },
    tools=[query_birthday_db]
)

# --- 2. Setup Tool Registry ---
birthday_tool_registry = {
    "query_birthday_db": _query_birthday_db_impl
}

# --- 3. Run the Agent ---
run_agent_turn(
    agent=birthday_agent,
    tool_registry=birthday_tool_registry,
    user_message="whens my bday????"
)


👤 User Message: whens my bday????
-----------------------------------------------------
🧠 Reasoning: The user's query requires using a tool to get the answer.
-----------------------------------------------------
🔧 Tool Call: query_birthday_db
{
  "name": "Sarah"
}
-----------------------------------------------------
🔧 Tool Return: 07-06-1993
-----------------------------------------------------
🧠 Reasoning: The tool has returned information. Now generating a final response for the user.
-----------------------------------------------------
🤖 Agent: Your birthday is July 6th, 1993.

-----------------------------------------------------
